## Errors and Exceptions


<p>No matter your skill as a programmer, you will eventually make a coding mistake.
Such mistakes come in three basic flavors:</p>
<ul>
<li><em><b>Syntax errors: </b></em> Errors where the code is not valid Python (generally easy to fix)</li>
<li><em><b>Runtime errors: </b></em> Errors where syntactically valid code fails to execute, perhaps due to invalid user input (sometimes easy to fix)</li>
<li><em><b>Semantic errors: </b></em> Errors in logic: code executes without a problem, but the result is not what you expect (often very difficult to track-down and fix)</li>
</ul>
<p>Here we're going to focus on how to deal cleanly with <em><b>runtime errors</b></em>.
As we'll see, Python handles runtime errors via its <em>exception handling</em> framework.</p>



<h2 id="Runtime-Errors">Runtime Errors<a class="anchor-link" href="#Runtime-Errors">¶</a></h2><p>If you've done any coding in Python, you've likely come across runtime errors.
They can happen in a lot of ways.</p>
<p>For example, if you try to reference an undefined variable:</p>


In [1]:
print(Q)

NameError: name 'Q' is not defined


<p>Or if you try an operation that's not defined:</p>


In [2]:
1 + 'abc'

TypeError: unsupported operand type(s) for +: 'int' and 'str'


<p>Or you might be trying to compute a mathematically ill-defined result:</p>


In [3]:
2 / 0

ZeroDivisionError: division by zero


<p>Or maybe you're trying to access a sequence element that doesn't exist:</p>


In [4]:
L = [1, 2, 3]
L[1000]

IndexError: list index out of range


<p>Note that in each case, Python is kind enough to not simply indicate that an error happened, but to spit out a <em>meaningful</em> exception that includes information about what exactly went wrong, along with the exact line of code where the error happened.
Having access to meaningful errors like this is immensely useful when trying to trace the root of problems in your code.</p>



<h2 id="Catching-Exceptions:-try-and-except">Catching Exceptions: <code>try</code> and <code>except</code><a class="anchor-link" href="#Catching-Exceptions:-try-and-except">¶</a></h2><p>The main tool Python gives you for handling runtime exceptions is the <code>try</code>...<code>except</code> clause.
Its basic structure is this:</p>


In [5]:
try:
    print("this gets executed first")
except:
    print("this gets executed only if there is an error")

this gets executed first



<p>Note that the second block here did not get executed: this is because the first block did not return an error.
Let's put a problematic statement in the <code>try</code> block and see what happens:</p>


In [6]:
try:
    print("let's try something:")
    x = 1 / 0 # ZeroDivisionError
except:
    print("something bad happened!")

let's try something:
something bad happened!



<p>Here we see that when the error was raised in the <code>try</code> statement (in this case, a <code>ZeroDivisionError</code>), the error was caught, and the <code>except</code> statement was executed.</p>
<p>One way this is often used is to check user input within a function or another piece of code.
For example, we might wish to have a function that catches zero-division and returns some other value, perhaps a suitably large number like $10^{100}$:</p>


In [7]:
def safe_divide(a, b):
    try:
        return a / b
    except:
        return 1E100

In [8]:
safe_divide(1, 2)

0.5

In [9]:
safe_divide(2, 0)

1e+100


<p>There is a subtle problem with this code, though: what happens when another type of exception comes up? For example, this is probably not what we intended:</p>


In [10]:
safe_divide (1, '2')

1e+100


<p>Dividing an integer and a string raises a <code>TypeError</code>, which our over-zealous code caught and assumed was a <code>ZeroDivisionError</code>!
For this reason, it's nearly always a better idea to catch exceptions <em>explicitly</em>:</p>


In [11]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 1E100

In [12]:
safe_divide(1, 0)

1e+100

In [13]:
safe_divide(1, '2')

TypeError: unsupported operand type(s) for /: 'int' and 'str'


<p>We're now catching zero-division errors only, and letting all other errors pass through un-modified.</p>



<h2 id="Raising-Exceptions:-raise">Raising Exceptions: <code>raise</code><a class="anchor-link" href="#Raising-Exceptions:-raise">¶</a></h2><p>We've seen how valuable it is to have informative exceptions when using parts of the Python language.
It's equally valuable to make use of informative exceptions within the code you write, so that users of your code (foremost yourself!) can figure out what caused their errors.</p>
<p>The way you raise your own exceptions is with the <code>raise</code> statement. For example:</p>


In [14]:
raise RuntimeError("my error message")

RuntimeError: my error message


<p>As an example of where this might be useful, let's return to our <code>fibonacci</code> function that we defined previously:</p>


In [15]:
def fibonacci(N):
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L


<p>One potential problem here is that the input value could be negative.
This will not currently cause any error in our function, but we might want to let the user know that a negative <code>N</code> is not supported.
Errors stemming from invalid parameter values, by convention, lead to a <code>ValueError</code> being raised:</p>


In [16]:
def fibonacci(N):
    if N < 0:
        raise ValueError("N must be non-negative")
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

In [17]:
fibonacci(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [18]:
fibonacci(-10)

ValueError: N must be non-negative


<p>Now the user knows exactly why the input is invalid, and could even use a <code>try</code>...<code>except</code> block to handle it!</p>


In [19]:
N = -10
try:
    print("trying this...")
    print(fibonacci(N))
except ValueError:
    print("Bad value: need to do something else")

trying this...
Bad value: need to do something else



<h2 id="Diving-Deeper-into-Exceptions">Diving Deeper into Exceptions<a class="anchor-link" href="#Diving-Deeper-into-Exceptions">¶</a></h2><p>Briefly, I want to mention here some other concepts you might run into.
I'll not go into detail on these concepts and how and why to use them, but instead simply show you the syntax so you can explore more on your own.</p>



<h3 id="Accessing-the-error-message">Accessing the error message<a class="anchor-link" href="#Accessing-the-error-message">¶</a></h3><p>Sometimes in a <code>try</code>...<code>except</code> statement, you would like to be able to work with the error message itself.
This can be done with the <code>as</code> keyword:</p>


In [20]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print("Error class is:  ", type(err))
    print("Error message is:", err)

Error class is:   <class 'ZeroDivisionError'>
Error message is: division by zero



<p>With this pattern, you can further customize the exception handling of your function.</p>



<h3 id="Defining-custom-exceptions">Defining custom exceptions<a class="anchor-link" href="#Defining-custom-exceptions">¶</a></h3><p>In addition to built-in exceptions, it is possible to define custom exceptions through <em>class inheritance</em>.
For instance, if you want a special kind of <code>ValueError</code>, you can do this:</p>


In [21]:
class MySpecialError(ValueError):
    pass

raise MySpecialError("here's the message")

MySpecialError: here's the message


<p>This would allow you to use a <code>try</code>...<code>except</code> block that only catches this type of error:</p>


In [22]:
try:
    print("do something")
    raise MySpecialError("[informative error message here]")
except MySpecialError:
    print("do something else")

do something
do something else



<p>You might find this useful as you develop more customized code.</p>



<h2 id="try...except...else...finally"><code>try</code>...<code>except</code>...<code>else</code>...<code>finally</code><a class="anchor-link" href="#try...except...else...finally">¶</a></h2><p>In addition to <code>try</code> and <code>except</code>, you can use the <code>else</code> and <code>finally</code> keywords to further tune your code's handling of exceptions.
The basic structure is this:</p>


In [23]:
try:
    print("try something here")
except:
    print("this happens only if it fails")
else:
    print("this happens only if it succeeds")
finally:
    print("this happens no matter what")

try something here
this happens only if it succeeds
this happens no matter what



<p>The utility of <code>else</code> here is clear, but what's the point of <code>finally</code>?
Well, the <code>finally</code> clause really is executed <em>no matter what</em>: I usually see it used to do some sort of cleanup after an operation completes.</p>
